# Trade Reconciliation Between Internal and Clearer Data
This assessment is designed to evaluate the skills and understanding of a candidate in reconciling trade data between internal records and clearer records. My solution will involve comparing two datasets, and follow the instructions provided in the brief:

1. Understand the structure of the datasets. 
   * Internal data and clearer data are provided in xlsx format. The internal data contains a trade for 2nd quarter which comprise of 2025-04, 2025-05, and 2025-06 months. In addition to this, 2025-04 & 2025-05 months also contains an individual trade. 
2. Decompose the quarter trades into individual month trades.
    * The quarter trades will be split into individual month trades for April, May, and June. The goal is to identically divide the quarter trades into individual month trades, and merge trades for 2025-04 and 2025-05 months, with avg price. 
3. Net Internal Positions <br>

4. Reconciliation of the net internal positions with the clearer data.
    * The reconciliation will involve comparing the net internal positions with the clearer data to identify any discrepancies. This will include checking for differences in quantities and prices, and resolving any mismatches.

## 1.1 Importing Required Libraries


In [1]:
# Setup and Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import Utilities
import sys
sys.path.insert(0, '..')  # Go up one directory to project root

from utils.data_loader import load_data
from utils.reconciliation import convert_qtr_into_months, net_monthly_trades, reconcile_trades, print_reconciliation_summary  # This is correct nowcorrect now

## 1.2 Loading the Data


In [2]:
pwd

'd:\\numustafa_trade_reconciliation\\notebook'

In [3]:
internal_data = load_data('../data/internal_trade_data.xlsx')
clearer_data = load_data('../data/clearer_trade_data.xlsx')


print(f"Internal Data Shape: {internal_data.shape}")
print(f"Clearer Data Shape: {clearer_data.shape}")

Internal Data Shape: (4, 5)
Clearer Data Shape: (4, 3)


In [4]:
df_internal = internal_data.copy()
df_clearer = clearer_data.copy()

In [5]:
display(df_internal)

,trade_id,contract_type,contract_period,quantity_mwh,price_eur_per_mwh
0,1,Outright,2025-03,100,30
1,2,Outright,2025-04,200,31
2,3,Outright,2025-05,250,32
3,4,Outright Quarter,Q225,300,33


In [6]:
display(df_clearer)

,delivery_month,quantity_mwh,price_eur_per_mwh
0,2025-03,100,30.0
1,2025-04,500,32.0
2,2025-05,550,32.5
3,2025-06,300,33.0


In [7]:
quarter_data = df_internal.iloc[3]
print(f"Quarter Data: {quarter_data}")

Quarter Data: trade_id                            4
contract_type        Outright Quarter
contract_period                  Q225
quantity_mwh                      300
price_eur_per_mwh                  33
Name: 3, dtype: object


In [8]:
convert = convert_qtr_into_months(quarter_data)
print(f"Converted Data: {convert}")

Converted Data:    trade_id         contract_type contract_period  quantity_mwh  \
0         1  Quarterly decomposed         2025-04         100.0   
1         2  Quarterly decomposed         2025-05         100.0   
2         3  Quarterly decomposed         2025-06         100.0   

   price_eur_per_mwh  
0                 33  
1                 33  
2                 33  


In [9]:
convert.dtypes

trade_id               int64
contract_type         object
contract_period       object
quantity_mwh         float64
price_eur_per_mwh      int64
dtype: object

In [10]:
# drop the last row
df_internal = df_internal.drop(df_internal.index[-1])
df_internal

,trade_id,contract_type,contract_period,quantity_mwh,price_eur_per_mwh
0,1,Outright,2025-03,100,30
1,2,Outright,2025-04,200,31
2,3,Outright,2025-05,250,32


In [11]:
# add the new monthly rows to the dataframe
df_internal_new = pd.concat([df_internal, convert], ignore_index=True)
df_internal_new 

,trade_id,contract_type,contract_period,quantity_mwh,price_eur_per_mwh
0,1,Outright,2025-03,100.0,30
1,2,Outright,2025-04,200.0,31
2,3,Outright,2025-05,250.0,32
3,1,Quarterly decomposed,2025-04,100.0,33
4,2,Quarterly decomposed,2025-05,100.0,33
5,3,Quarterly decomposed,2025-06,100.0,33


In [12]:
df_internal_proper = net_monthly_trades(df_internal_new)
print(f"Internal Data Proper Shape: {df_internal_proper.shape}")
df_internal_proper

Internal Data Proper Shape: (4, 4)


,contract_type,contract_period,quantity_mwh,price_eur_per_mwh
0,Net Monthly Trade,2025-03,100,30.0
1,Net Monthly Trade,2025-04,300,31.7
2,Net Monthly Trade,2025-05,350,32.3
3,Net Monthly Trade,2025-06,100,33.0


In [13]:
df_clearer

,delivery_month,quantity_mwh,price_eur_per_mwh
0,2025-03,100,30.0
1,2025-04,500,32.0
2,2025-05,550,32.5
3,2025-06,300,33.0


In [14]:
df_internal_proper.dtypes


contract_type         object
contract_period       object
quantity_mwh           int64
price_eur_per_mwh    float64
dtype: object

In [15]:
df_clearer.dtypes

delivery_month        object
quantity_mwh           int64
price_eur_per_mwh    float64
dtype: object

In [16]:
# rename months to the same column name
df_int  = df_internal_proper.rename(columns={"contract_period": "month"})
df_clr  = df_clearer.rename(columns={"delivery_month": "month"})

In [17]:
df_int.dtypes

contract_type         object
month                 object
quantity_mwh           int64
price_eur_per_mwh    float64
dtype: object

In [18]:
df_clr.dtypes

month                 object
quantity_mwh           int64
price_eur_per_mwh    float64
dtype: object

In [19]:
# Merge the two dataframes on the month column
new = reconcile_trades(df_int, df_clr)
print(f"Reconciled Data Shape: {new.shape}")
new 

Reconciled Data Shape: (4, 7)


,month,quantity_mwh_internal,quantity_mwh_clearer,quantity_diff,price_eur_per_mwh_internal,price_eur_per_mwh_clearer,price_diff
0,2025-03,100,100,0,30.0,30.0,0.0
1,2025-04,300,500,-200,31.7,32.0,-0.3
2,2025-05,350,550,-200,32.3,32.5,-0.2
3,2025-06,100,300,-200,33.0,33.0,0.0


In [20]:
# Summary of the reconciliation
print_reconciliation_summary(new)


Reconciliation Summary:
     month  quantity_mwh_internal  quantity_mwh_clearer  quantity_diff  \
0  2025-03                    100                   100              0   
1  2025-04                    300                   500           -200   
2  2025-05                    350                   550           -200   
3  2025-06                    100                   300           -200   

   price_eur_per_mwh_internal  price_eur_per_mwh_clearer  price_diff  
0                        30.0                       30.0         0.0  
1                        31.7                       32.0        -0.3  
2                        32.3                       32.5        -0.2  
3                        33.0                       33.0         0.0  

Total Trades Reconciled: 4
Total Quantity Mismatches: 600

Total Quantity Difference: -600
Total Price Difference: -0.5

Reconciliation completed successfully.
